# Este notebook destina-se à geração do modelo utilizando os dados processados conforme o notebook processa_dados


## Gerando o modelo

In [0]:
Montando o Google Drive

In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Carregando dados processados

In [0]:
def carregar_dados_pickle(path):
    import pickle
    with open(path, "rb") as infile:
        return pickle.load(infile)

In [0]:
token2int, int2token = carregar_dados_pickle("drive/My Drive/text_sumarizer/data/token2int_int2token.pickle")
token2vetor = carregar_dados_pickle("drive/My Drive/text_sumarizer/data/token2vetor.pickle")
int_titulos_ord, int_textos_ord = carregar_dados_pickle("drive/My Drive/text_sumarizer/data/int_titulos_textos_ord.pickle")
matriz_de_tokens = carregar_dados_pickle("drive/My Drive/text_sumarizer/data/matriz_de_tokens.pickle")

Será usado o TensorFlow para geração do modelo

In [6]:
import tensorflow as tf
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('Versão do TensorFlow: {}'.format(tf.__version__))

Versão do TensorFlow: 1.14.0-rc1


In [0]:
import time
import numpy as np

In [5]:
nome_disp = tf.test.gpu_device_name()
if nome_disp != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('GPU encontrada em: {}'.format(nome_disp))

GPU encontrada em: /device:GPU:0


Definindo uma função para criar placeholders para os dados de entrada do modelo

In [0]:
def entradas_do_modelo():
    #Cria placeholders para os dados de entrada
    
    dados_de_entrada = tf.placeholder(tf.int32, [None, None], name='entrada')
    alvos = tf.placeholder(tf.int32, [None, None], name='alvos')
    ta = tf.placeholder(tf.float32, name='taxa_aprend')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    tam_sumario = tf.placeholder(tf.int32, (None,), name='tam_sumario')
    tam_max_sumario = tf.reduce_max(tam_sumario, name='tam_max_dec')
    tam_texto = tf.placeholder(tf.int32, (None,), name='tam_texto')

    return dados_de_entrada, alvos, ta, keep_prob, tam_sumario, tam_max_sumario, tam_texto

In [0]:
def codificar_entrada(dados_de_destino, token2int, tam_batch):
    """
    Remove o último id de token de cada batch e concatena
    o <GO> ao início de cada batch    
    """
    
    final = tf.strided_slice(dados_de_destino, [0, 0], 
                             [tam_batch, -1], [1, 1])
    entrada_dec = tf.concat([tf.fill([tam_batch, 1], 
                             token2int['<GO>']), final], 1)

    return entrada_dec

In [0]:
 def camada_de_codificacao(tam_rnn, tam_sequencia, num_camadas, entradas_rnn, keep_prob):
    """
    Cria a camada de codificação
    """

    for camada in range(num_camadas):
        with tf.variable_scope('codificador_{}'.format(camada)):
            celula_fw = tf.contrib.rnn.LSTMCell(tam_rnn,
                        initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            celula_fw = tf.contrib.rnn.DropoutWrapper(celula_fw, 
                        input_keep_prob = keep_prob)

            celula_bw = tf.contrib.rnn.LSTMCell(tam_rnn,
                        initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            celula_bw = tf.contrib.rnn.DropoutWrapper(celula_bw, 
                        input_keep_prob = keep_prob)

            saida_cod, estado_de_cod = tf.nn.bidirectional_dynamic_rnn(celula_fw, 
                                        celula_bw, entradas_rnn, tam_sequencia, 
                                            dtype=tf.float32)

    # Junta as entradas porque estamos usando uma RNN bidirecional
    saida_cod = tf.concat(saida_cod,2)

    return saida_cod, estado_de_cod

In [0]:
def treina_camada_decod(vetor_entrada_dec, tam_sumario, celula_dec, estado_inicial,
                            camada_saida, tam_vocab, tam_max_sumario):
    
        """Cria os treinadores logits"""
    
        treinador_auxiliar = tf.contrib.seq2seq.TrainingHelper(inputs=vetor_entrada_dec,
                            sequence_length=tam_sumario, time_major=False)

        treinador_decodificador = tf.contrib.seq2seq.BasicDecoder(celula_dec,
                            treinador_auxiliar, estado_inicial,camada_saida) 

        treinador_logits, _ , _ = tf.contrib.seq2seq.dynamic_decode(treinador_decodificador,
                                    output_time_major=False, impute_finished=True,
                                        maximum_iterations=tam_max_sumario)
        return treinador_decodificador

In [0]:
def camada_de_decod_inferencial(vetores, token_inicial, token_final, celula_decod, 
                                    estado_inicial, camada_de_saida, tam_max_sumario, batch_size):
        
        """
        Cria logits inferenciais
        """
        
        tokens_iniciais = tf.tile(tf.constant([token_inicial], dtype=tf.int32), 
                                  [batch_size], name='tokens_iniciais')
        
        aux_inferencial = tf.contrib.seq2seq.GreedyEmbeddingHelper(vetores,
                            tokens_iniciais, token_final)
                    
        decod_inferencial = tf.contrib.seq2seq.BasicDecoder(celula_decod,
                                aux_inferencial, estado_inicial, camada_de_saida)
                    
        logits_inferencial, _ , _ = tf.contrib.seq2seq.dynamic_decode(decod_inferencial,
                                        output_time_major=False, impute_finished=True,
                                            maximum_iterations=tam_max_sumario)
        
        return decod_inferencial

In [0]:
def camada_de_decodificacao(vetor_entrada_dec, vetores, saida_cod, estado_de_cod, tam_vocab,
                tam_texto, tam_sumario, tam_max_sumario, tam_rnn, token2int, keep_prob, 
                    tam_batch, num_camadas):

    """
    Cria a célula de decodificação e atenção para o treinamento e camadas 
    decodificadoras inferenciais
    """
    
    for camada in range(num_camadas):
        with tf.variable_scope('decodificador_{}'.format(camada)):
            lstm = tf.contrib.rnn.LSTMCell(tam_rnn,
                    initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            celula_decod = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)

    camada_de_saida = Dense(tam_vocab, kernel_initializer= 
                        tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))

    mecan_atencao = tf.contrib.seq2seq.BahdanauAttention(tam_rnn,
                    saida_cod, tam_texto, normalize=False, name='BahdanauAttention')

    celula_decod = tf.contrib.seq2seq.AttentionWrapper(celula_decod, mecan_atencao, tam_rnn)

    #estado_inicial = tf.contrib.seq2seq.AttentionWrapperState(estado_de_cod[0],
    #                   _zero_state_tensors(tam_rnn, batch_size, tf.float32)) 

    estado_inicial = celula_decod.zero_state(batch_size=batch_size, 
                        dtype=tf.float32).clone(cell_state=estado_de_cod[0])

    with tf.variable_scope("decode"):
        treinador_decodificador = treina_camada_decod(vetor_entrada_dec, tam_sumario, 
                                celula_decod, estado_inicial, camada_de_saida, tam_vocab, tam_max_sumario)

        treinador_logits,_ ,_ = tf.contrib.seq2seq.dynamic_decode(treinador_decodificador,
                                    output_time_major=False, impute_finished=True,
                                        maximum_iterations=tam_max_sumario)

    with tf.variable_scope("decode", reuse=True): inference_decoder= \
                    camada_de_decod_inferencial(vetores, token2int['<GO>'], 
                        token2int['<EOS>'], celula_decod, estado_inicial, 
                            camada_de_saida, tam_max_sumario,  tam_batch)


    logits_inferencial,_ ,_ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                            output_time_major=False,
                            impute_finished=True,
                            maximum_iterations=tam_max_sumario)

    return treinador_logits, logits_inferencial

In [0]:
def modelo_seq2seq(dados_de_entrada, dados_de_destino, keep_prob, tam_texto, tam_sumario, tam_max_sumario, 
                  tam_vocab, tam_rnn, num_camadas, token2int, tam_batch):
    """
    Usa as funções anteriores para criar os logits de treinamento inferenciais
    """
    
    # Usa os vetores do modelo GloVe e os novos gerados aleatoriamente
    vetores = matriz_de_tokens
    
    vetores_ent_codif = tf.nn.embedding_lookup(vetores, dados_de_entrada)
    saida_cod, estado_de_cod = camada_de_codificacao(tam_rnn, tam_texto, num_camadas, vetores_ent_codif, keep_prob)
    
    saida_decod = codificar_entrada(dados_de_destino, token2int, tam_batch)
    vetor_entrada_dec = tf.nn.embedding_lookup(vetores, saida_decod)
    
    treinador_logits, logits_inferencial  = camada_de_decodificacao(vetor_entrada_dec, 
                                             vetores, saida_cod, estado_de_cod, 
                                                tam_vocab, tam_texto, tam_sumario, 
                                                    tam_max_sumario, tam_rnn, token2int, 
                                                        keep_prob, tam_batch, num_camadas)
    
    return treinador_logits, logits_inferencial

In [0]:
def preenche_grupo_de_sentencas(grupo_sents):
    """
    Preenche as sentenças com <PAD> para que cada sentença de 
    um grupo de sentenças tenha o mesmo tamanho
    """

    tam_max_sentenca = max([len(sentenca) for sentenca in grupo_sents])
    return [sentenca + [token2int['<PAD>']] * (tam_max_sentenca - \
                len(sentenca)) for sentenca in grupo_sents]

In [0]:
def get_batches(sumarios, textos, batch_size):
    """
    Agrupa sumarios, textos, e os tamanhos de suas sentenças
    """
    
    for grupo_i in range(0, len(textos)//batch_size):
        inicio_i = grupo_i * batch_size
        grupo_de_sumarios = sumarios[inicio_i:inicio_i + batch_size]
        grupo_de_textos = textos[inicio_i:inicio_i + batch_size]
        preenche_grupo_de_sumarios = np.array(preenche_grupo_de_sentencas(grupo_de_sumarios))
        preench_grupo_textos = np.array(preenche_grupo_de_sentencas(grupo_de_textos))
        
        # Need the lengths for the _lengths parameters
        tams_preench_sums = []
        for sumario in preenche_grupo_de_sumarios:
            tams_preench_sums.append(len(sumario))
        
        tams_preench_textos = []
        for text in preench_grupo_textos:
            tams_preench_textos.append(len(text))
        
        yield preenche_grupo_de_sumarios, preench_grupo_textos, tams_preench_sums, tams_preench_textos

In [0]:
# Seta os Hyperparametros
epochs = 100
batch_size = 64
tam_rnn = 256
num_camadas = 2
taxa_aprend = 0.005
keep_probab = 0.75

In [16]:
# Constrói o Grafo
grafo_de_treino = tf.Graph()
# Seta o Grafo para o padrão para garantir que ele está pronto
with grafo_de_treino.as_default():
    
    # Carrega as entradas do modelo    
    dados_de_entrada, alvos, ta, keep_prob, tam_sumario, tam_max_sumario, tam_texto = entradas_do_modelo()

    # Cria os treinadores logits e inferencial
    treinador_logits, logits_inferencial = modelo_seq2seq(tf.reverse(dados_de_entrada, [-1]),
                                            alvos, keep_prob, tam_texto, tam_sumario,
                                                tam_max_sumario, len(token2int)+1, tam_rnn, 
                                                    num_camadas, token2int, batch_size)

    # Cria os tensores para os treinadores logits e logits inferenciais
    treinador_logits = tf.identity(treinador_logits.rnn_output, 'logits')
    logits_inferencial = tf.identity(logits_inferencial.sample_id, name='predicoes')

    # Cria os pesos para o "sequence_loss"
    mascaras = tf.sequence_mask(tam_sumario, tam_max_sumario, dtype=tf.float32, name='mascaras')

    with tf.name_scope("otimizacao"):
        # Função "Loss"
        custo = tf.contrib.seq2seq.sequence_loss(
            treinador_logits,
            alvos,
            mascaras)

        # Otimizador
        otimizador = tf.train.AdamOptimizer(taxa_aprend)

        # Limitação do gradiente
        gradientes = otimizador.compute_gradients(custo)
        gradientes_limitados = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradientes if grad is not None]
        train_op = otimizador.apply_gradients(gradientes_limitados)

print("O Grafo foi construido.")

W0624 20:12:39.630298 140345706928000 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0624 20:12:39.632451 140345706928000 deprecation.py:323] From <ipython-input-8-384285c4ae8e>:9: LSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0624 20:12:39.646617 140345706928000 deprecation.py:323] From <ipython-input-8-384285c4ae8e>:20: bidirectional_dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updatin

O Grafo foi construido.


# Treinando o Modelo

Definindo um subconjunto do dataset para facilitar treinamentos de teste

In [3]:
# Subconjunto de dados de treinamento
inicio = 3000
fim = inicio + 50000
porcao_sumarios_ord = int_titulos_ord[inicio:fim]
porcao_textos_ord = int_textos_ord[inicio:fim]
print("Tamanho do menor texto na porção:", len(porcao_textos_ord[0]))
print("Tamanho do maior texto na porção:", len(porcao_textos_ord[-1]))

NameError: ignored

In [26]:
# Treina o modelo
decaimento_taxa_aprend = 0.95
taxa_aprend_min = 0.0005
mostrar_intervalo = 20 # Verifica "training loss" a cada 20 "batches"
parar_antes = 0 
parar = 6 #3 # Se o loss não diminuir em 3 verificações, pare o treinamento
por_epoch = 3 # Faça 3 verificações de atualização por epoch
verific_atualizacao = (len(porcao_textos_ord)//batch_size//por_epoch)-1

atualiz_loss = 0 
batch_loss = 0
atualiz_loss_sumario = [] # Armazendo as atulizacões dos losses para salvar melhoras no modelo

  
tf.reset_default_graph()
checkpoint = "drive/My Drive/text_sumarizer/models/sum/best_model.ckpt"
with tf.Session(graph=grafo_de_treino) as sess:
    sess.run(tf.global_variables_initializer())
    
    # Se quisermos continuar treinando uma sessão anterior
    # loader = tf.train.import_meta_graph(checkpoint + '.meta')
    # loader.restore(sess, checkpoint)
    # sess.run(tf.local_variables_initializer())

    for epoch_i in range(1, epochs+1):
        atualiz_loss = 0
        batch_loss = 0
        for batch_i, (batch_sumarios, batch_textos, tams_sumarios, tams_textos) in enumerate(
                get_batches(porcao_sumarios_ord, porcao_textos_ord, batch_size)):
            tempo_inicio = time.time()
            _, loss = sess.run(
                [train_op, custo],
                {dados_de_entrada: batch_textos,
                 alvos: batch_sumarios,
                 ta: taxa_aprend,
                 tam_sumario: tams_sumarios,
                 tam_texto: tams_textos,
                 keep_prob: keep_probab})

            batch_loss += loss
            atualiz_loss += loss
            tempo_fin = time.time()
            tempo_batch = tempo_fin - tempo_inicio

            if batch_i % mostrar_intervalo == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(porcao_textos_ord) // batch_size, 
                              batch_loss / mostrar_intervalo, 
                              tempo_batch*mostrar_intervalo))
                batch_loss = 0
                
                #saver = tf.train.Saver() 
                #saver.save(sess, checkpoint)
                
            if batch_i % verific_atualizacao == 0 and batch_i > 0:
                print("Perda média dessa atualização:", round(atualiz_loss/verific_atualizacao,3))
                atualiz_loss_sumario.append(atualiz_loss)
                
              
                  
                # Se a perda da atualização estiver em um novo mínimo, salve o modelo
                if atualiz_loss <= min(atualiz_loss_sumario):
                    print('Novo recorde!') 
                    parar_antes = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("Sem melhora.")
                    parar_antes += 1
                    if parar_antes == parar:
                        break
                atualiz_loss = 0
            
                    
        # Reduz a taxa de aprendizado, mas não abaixo do seu valor mínimo
        taxa_aprend *= decaimento_taxa_aprend
        if taxa_aprend < taxa_aprend_min:
            taxa_aprend = taxa_aprend_min
        
        if parar_antes == parar:
            print("Parando o treinamento.")
            break



Perda média dessa atualização: 20.175
Novo recorde!
Perda média dessa atualização: 21.496
Sem melhora.
Perda média dessa atualização: 7.531
Novo recorde!
Perda média dessa atualização: 10.228
Sem melhora.
Perda média dessa atualização: 8.367
Sem melhora.
Perda média dessa atualização: 6.727
Novo recorde!


In [19]:
checkpoint = "drive/My Drive/text_sumarizer/models/sum/best_model.ckpt" 

grafo_carregado = tf.Graph()
with tf.Session(graph=grafo_carregado) as sess:
    # Carrega o modelo salvo
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)
    names = []
    [names.append(n.name) for n in grafo_carregado.as_graph_def().node]
names

W0624 20:09:13.655901 139855912085376 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


['entrada',
 'alvos',
 'taxa_aprend',
 'keep_prob',
 'tam_sumario',
 'Const',
 'tam_max_dec',
 'tam_texto',
 'ReverseV2/axis',
 'ReverseV2',
 'embedding_lookup/params_0',
 'embedding_lookup/axis',
 'embedding_lookup',
 'embedding_lookup/Identity',
 'codificador_0/DropoutWrapperInit/Const',
 'codificador_0/DropoutWrapperInit/Const_1',
 'codificador_0/DropoutWrapperInit_1/Const',
 'codificador_0/DropoutWrapperInit_1/Const_1',
 'codificador_0/bidirectional_rnn/fw/fw/Rank',
 'codificador_0/bidirectional_rnn/fw/fw/range/start',
 'codificador_0/bidirectional_rnn/fw/fw/range/delta',
 'codificador_0/bidirectional_rnn/fw/fw/range',
 'codificador_0/bidirectional_rnn/fw/fw/concat/values_0',
 'codificador_0/bidirectional_rnn/fw/fw/concat/axis',
 'codificador_0/bidirectional_rnn/fw/fw/concat',
 'codificador_0/bidirectional_rnn/fw/fw/transpose',
 'codificador_0/bidirectional_rnn/fw/fw/sequence_length',
 'codificador_0/bidirectional_rnn/fw/fw/Shape',
 'codificador_0/bidirectional_rnn/fw/fw/strided_sl

# Gerando sumários utilizando o modelo

In [0]:
import re
def limpar_texto(texto, remover_stopwords=False):
    
    # Converte o texto para lower case
    texto = texto.lower()
    
    # Remove caracteres indesejados
    texto = re.sub(r'https?:\/\/.*[\r\n]*', '', texto, flags=re.MULTILINE)
    texto = re.sub(r'\<a href', ' ', texto)
    texto = re.sub(r'&amp;', '', texto) 
    texto = re.sub(r'[_"\%()|+&=*%!?:#$@\[\]/]', ' ', texto)
    texto = re.sub(r'<br />', ' ', texto)
    texto = re.sub(r'\'', ' ', texto)
    
    # Opcionalmente, remove stop words
    if remover_stopwords:
        tokens = [t for t in normalizer.tokenize_words(texto)
                    if t not in STOPWORDS]
        texto = detokenizer.detokenize(tokens, return_str=True)
        
    return texto

Carregando textos do dataset para serem usados nos testes

In [0]:
titulos, textos = carregar_dados_pickle("drive/My Drive/text_sumarizer/data/titulos_e_textos.pickle")

In [0]:
def usr_texto2seq(texto):
    '''Prepara o texto para o modelo'''
    
    texto = limpar_texto(texto)
    return [token2int.get(token, token2int['<UNK>']) 
            for token in texto.split()]

In [11]:
indice_aleatorio = np.random.randint(0,len(int_textos_ord))
texto_entrada = textos[indice_aleatorio]
int_texto = usr_texto2seq(textos[indice_aleatorio])

checkpoint = "drive/My Drive/text_sumarizer/models/sum/best_model.ckpt"

grafo_carregado = tf.Graph()
with tf.Session(graph=grafo_carregado) as sess:
    # Carrega o modelo salvo
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    dados_entrada = grafo_carregado.get_tensor_by_name('entrada:0')
    logits = grafo_carregado.get_tensor_by_name('predicoes:0')
    tam_texto = grafo_carregado.get_tensor_by_name('tam_texto:0')
    tam_sumario = grafo_carregado.get_tensor_by_name('tam_sumario:0')
    keep_prob = grafo_carregado.get_tensor_by_name('keep_prob:0')
    
    # Multiplica por batch_size para ficar igual aos parâmetros de entrada do modelo
    resposta_logits = sess.run(logits, {dados_entrada: [int_texto]*batch_size, 
                                      tam_sumario: [np.random.randint(5,8)], 
                                      tam_texto: [len(int_texto)]*batch_size,
                                      keep_prob: 1.0})[0] 

# Remove o pad
pad = token2int["<PAD>"] 

print("Texto original:", textos[indice_aleatorio])
print("Sumário original:", titulos[indice_aleatorio])

print("\nTexto")
print(f"  Ids dos tokens:    {[i for i in int_texto]}")
print(f"  Tokens de entrada: {[int2token[i] for i in int_texto]}")

print("\nSumario")
print(f"  Ids dos tokens:    {[i for i in resposta_logits if i != pad]}")
print(f"  Resposta textual:  {[int2token[i] for i in resposta_logits if i != pad]}")

W0624 20:34:50.006259 140348186097536 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Texto original: chama beta, chama melo. tome gêmeos, tamanha semelhança física brinque jogo sete erros foto desta página, demi moore. musa irmão, diz pernambucana roberta britto, 49 anos, dois menos romero. pinta. várias telas. melo tira beta sério vezes. raiva porque é família, famílias brigam, diz folha sentada poltrona romero britto, venda r 7.000 galeria roberta britto, rua oscar freire, paulo. lá vende obras irmão artistas trabalham material orgânico pop art. livro ainda título, caçula nove irmãos 49 69 anos narra história clã britto –e não poupa passagens mais cinzentas obra familiar famoso. 200 páginas escritas, inseriu críticas suposto descaso romero câncer matou mãe 2012, 89 anos. não. nenhum irmão trouxe fralda sequer pra mãe, reclama josé antônio, romulo, rosemiro, robson, roberval, risoleta romero. nono irmão, austriclinio, morreu. meio porta-voz, artista responde é muito triste pessoas sugerem contrário, sido sempre generoso família,, enquanto puder, continuarei sê-lo. é f